# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


In [42]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pickle
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
%load_ext autotime

from aux_func import evaluate_model, cargar_modelo

time: 0 ns (started: 2021-12-18 18:22:10 +01:00)


In [2]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

In [3]:
#Cargamos pipeline preprocesado
preprocessor = cargar_modelo('../models/preprocessor.pickle')

# Random Forest

Estos modelos surgen como una alternativa a los árboles de decisión, y el problema que tienen cuando se usan de manera individual. Al utilizar un árbol de decisión, los resultados no son consistentes, y pueden variar en función de cambios en la composición de los conjuntos de entrenamiento y validación.

El Random Forest utiliza muchos árboles, y la decisión final sobre una predicción concreta se basa en un sistema de voto. Este sistema se puede modificar, para ponderar el peso de los votos de los árboles en función de, por ejemplo, su precisión o el ROC-AUC Score (Área debajo de la curva ROC)

Cada árbol realiza su partición en conjunto de train y test, y se ejecuta. No hay ningún tipo de regla que influya en qué observaciones puede usar un algortimo, son independientes de las que haya utilizado otro arbol

Procedemos a entrenar el modelo.

![Highway](https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG)

In [4]:
clf = Pipeline(steps=[
    ('preprocesador', preprocessor), 
    
    ('clasificador', RandomForestClassifier(n_jobs=-1, random_state=0))])

In [5]:
%%time
clf.fit(xtrain, ytrain)

Wall time: 5min 52s


Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

In [13]:
with open('../models/RandomForest.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [14]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/RandomForest.pickle', 'rb') as f:
    clf = pickle.load(f)

Generamos las predicciones sobre los datos de validación y evaluamos el modelo.

In [10]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8140237379943902
Accuracy of the model: 0.9850499300081134

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    799946
           1       0.61      0.03      0.06     12291

    accuracy                           0.99    812237
   macro avg       0.80      0.52      0.53    812237
weighted avg       0.98      0.99      0.98    812237


Confusion matrix: 
[[799694    252]
 [ 11891    400]]



## Ajuste del umbral de predicción

Al observar cómo nuestro modelo obtiene un valor de recall muy bajo para la clase minoritaria, nos planteamos ajustar el umbral de predicción siguiendo el punto que obtiene la mayor G-mean o media geométrica entre las tasas de "true positives" y "false positives". La curva ROC nos permite determinar este punto, como observaremos más adelante.

In [15]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.016667, G-Mean=0.753
ROC-AUC score of the model: 0.8140237379943902
Accuracy of the model: 0.767305601690147

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.77      0.87    799946
           1       0.05      0.74      0.09     12291

    accuracy                           0.77    812237
   macro avg       0.52      0.75      0.48    812237
weighted avg       0.98      0.77      0.85    812237


Confusion matrix: 
[[614163 185783]
 [  3220   9071]]



Podemos observar como el ajuste del threshold dota al modelo de un mayor recall para los casos de la clase minoritaria, lo cual nos interesa desde un punto de vista práctico a pesar de reducir la precisión y accuracy del modelo. 

## Comprobación de overfitting

Comprobamos si el modelo sufre de overfitting, realizando una predicción sobre la serie de entrenamiento.

In [16]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.214310, G-Mean=1.000
ROC-AUC score of the model: 0.9999989972620618
Accuracy of the model: 0.9997620766125619

Classification report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   3200049
           1       0.98      1.00      0.99     48896

    accuracy                           1.00   3248945
   macro avg       0.99      1.00      1.00   3248945
weighted avg       1.00      1.00      1.00   3248945


Confusion matrix: 
[[3199290     759]
 [     14   48882]]



En efecto, el model Random Forest ha realizado un ajuste bastante notable. Posteriormente profundizaremos en este asunto, pero podemos afirmar que el modelo no generaliza bien.

# Random-Forest con SMOTE

En los modelos Random Forest y XGBoost también hemos realizado una prueba con una versión de los datos en los que realizamos un Oversample de la clase minoritaria mediante el algoritmo SMOTE. Probaremos esta técnica en estos primeros dos modelos más complejos con el objetivo de ver si merece la pena. Si consideramos que los resultados son peores a los obtenidos con los datos sin Oversample, nos mantendremos en la senda de usar los datos sin Oversample.

In [43]:
clf = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('smote', SMOTE(sampling_strategy=0.4, n_jobs=-1)),
    ('clasificador', RandomForestClassifier(n_jobs=-1, random_state=0))])

time: 0 ns (started: 2021-12-18 18:24:18 +01:00)


In [44]:
clf.fit(xtrain, ytrain)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 8min 31s (started: 2021-12-18 18:24:21 +01:00)


In [45]:
with open('../models/RandomForest_smote.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 3.78 s (started: 2021-12-18 18:32:53 +01:00)


In [46]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/RandomForest_smote.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 2.48 s (started: 2021-12-18 18:32:57 +01:00)


In [47]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8150212650032423
Accuracy of the model: 0.9848948028715757

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    799946
           1       0.51      0.05      0.08     12291

    accuracy                           0.98    812237
   macro avg       0.75      0.52      0.54    812237
weighted avg       0.98      0.98      0.98    812237


Confusion matrix: 
[[799403    543]
 [ 11726    565]]

time: 26.1 s (started: 2021-12-18 18:32:59 +01:00)


El modelo sin aplicar el ajuste de threshold no es lo suficientemente descriptivo como para poder compararlo con su versión análoga. Procedemos a realizar dicho ajuste.

## Ajuste del umbral de predicción

In [48]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)


Best Threshold=0.021480, G-Mean=0.749
ROC-AUC score of the model: 0.8150212650032423
Accuracy of the model: 0.76501562967459

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.77      0.87    799946
           1       0.05      0.73      0.09     12291

    accuracy                           0.77    812237
   macro avg       0.52      0.75      0.48    812237
weighted avg       0.98      0.77      0.85    812237


Confusion matrix: 
[[612370 187576]
 [  3287   9004]]

time: 1.45 s (started: 2021-12-18 18:33:26 +01:00)


Tras realizar el ajuste del modelo, podemos afirmar que en nuestro caso concreto y para el tratamiento de los datos que hemos realizado, no existe una razón por la que usar la versión Oversample de los datos.
Obtenemos una ligera diferencia en los resultados, obteniendo menor recall para la clase minoritaria. Comprobaremos si ocurre lo mismo en el modelo XGBoost.